# Whole Slide Data Tutorial & Examples

The wholeslidedata package aims to provide the tools to work with whole-slide images and annotations from different vendors and annotation software. The main contribution is a batch iterator that enables users to sample patches from the data efficiently, fast, and easily.

In [ ]:
#install the whole slide data package
#!pip install git+https://github.com/DIAGNijmegen/pathology-whole-slide-data@main

In [ ]:
import os
print(f"Previous exec path: {os.getcwd()}")
# move two level up
os.chdir('../../')
print(f"Current exec path: {os.getcwd()}")